In [3]:
!pip install gtsam==4.1.1.dev3

  Using cached gtsam-4.1.1.dev3-py3-none-macosx_10_9_x86_64.whl (20.4 MB)
Reason for being yanked: <none given>
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.9/bin/python3.9 -m pip install --upgrade pip' command.


In [31]:
import pandas as pd
# import gtsam
import sys, os
import yaml

sys.path.append("/Users/tushar.vaidya/codes/Python-VO/")

from utils.tools import plot_keypoints

from DataLoader import create_dataloader
from Detectors import create_detector
from Matchers import create_matcher

import cv2

In [5]:
traj = pd.read_csv("../results/kitti_superpoint_supergluematch.txt",header=None, delimiter=" ",names=["id","x","y","z","roll","pitch","yaw"])

In [ ]:
traj

In [6]:

matcher = create_matcher(config["matcher"])

NameError: name 'config' is not defined

In [10]:
with open("../params/custom_sift_flann_match.yaml", 'r') as f:
        config = yaml.load(f)

/Users/tushar.vaidya/py37/lib/python3.7/site-packages/ipykernel_launcher.py:2: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  


In [13]:
config

{'dataset': {'name': 'SequenceImageLoader',
  'root_path': '/Users/tushar.vaidya/datasets/kitti/hiring-assignment-lt/',
  'img_folder': 'images',
  'start': 0,
  'format': 'png'},
 'detector': {'name': 'HandcraftDetector',
  'type': 'SIFT',
  'SIFT': {'nfeatures': 1000,
   'nOctaveLayers': 3,
   'contrastThreshold': 0.04,
   'edgeThreshold': 10,
   'sigma': 1.6}},
 'matcher': {'name': 'FrameByFrameMatcher',
  'type': 'FLANN',
  'FLANN': {'kdTrees': 5, 'searchChecks': 50},
  'distance_ratio': 0.75}}

In [15]:
detector = create_detector(config["detector"])
# create matcher
matcher = create_matcher(config["matcher"])

In [17]:
loader = create_dataloader(config["dataset"])

In [19]:
img1 = loader[0]
img2 = loader[2]

In [24]:
kptdesc1 = detector(img1)
kptdesc2 = detector(img2)

In [26]:
pair = {}
pair["cur"] = kptdesc1
pair["ref"] =kptdesc2

In [28]:
matches = matcher(pair)

In [30]:
# matches

In [34]:
## refer linke for more functions https://www.programcreek.com/python/example/110665/cv2.triangulatePoints
def triangulate_points(self, kps_left, desps_left, kps_right, desps_right):
        matches = self.feature.row_match(
            kps_left, desps_left, kps_right, desps_right)
        assert len(matches) > 0

        px_left = np.array([kps_left[m.queryIdx].pt for m in matches])
        px_right = np.array([kps_right[m.trainIdx].pt for m in matches])

        points = cv2.triangulatePoints(
            self.left.projection_matrix, 
            self.right.projection_matrix, 
            px_left.transpose(), 
            px_right.transpose() 
            ).transpose()  # shape: (N, 4)

        points = points[:, :3] / points[:, 3:]

        can_view = np.logical_and(
            self.left.can_view(points), 
            self.right.can_view(points))

        mappoints = []
        matchs = []
        for i, point in enumerate(points):
            if not can_view[i]:
                continue
            normal = point - self.position
            normal = normal / np.linalg.norm(normal)

            color = self.left.get_color(px_left[i])

            mappoint = MapPoint(
                point, normal, desps_left[matches[i].queryIdx], color)
            mappoints.append(mappoint)
            matchs.append((matches[i].queryIdx, matches[i].trainIdx))

        return mappoints, matchs 

In [35]:
def triangulate_normalized_points(pose_1w, pose_2w, kpn_1, kpn_2):
    # P1w = np.dot(K1,  M1w) # K1*[R1w, t1w]
    # P2w = np.dot(K2,  M2w) # K2*[R2w, t2w]
    # since we are working with normalized coordinates x_hat = Kinv*x, one has         
    P1w = pose_1w[:3,:] # [R1w, t1w]
    P2w = pose_2w[:3,:] # [R2w, t2w]

    point_4d_hom = cv2.triangulatePoints(P1w, P2w, kpn_1.T, kpn_2.T)
    good_pts_mask = np.where(point_4d_hom[3]!= 0)[0]
    point_4d = point_4d_hom / point_4d_hom[3] 
    
    if __debug__:
        if False: 
            point_reproj = P1w @ point_4d;
            point_reproj = point_reproj / point_reproj[2] - add_ones(kpn_1).T
            err = np.sum(point_reproj**2)
            print('reproj err: ', err)     

    #return point_4d.T
    points_3d = point_4d[:3, :].T
    return points_3d, good_pts_mask  
